This is a tutorial to use the flip package: https://github.com/corentinravoux/flip \
It is self-contained and can be used in google collab or on your environement \
All the data used are subsampled version of a simulation. \
The data size is small for the tutorial, do not use it for science case.

In [1]:
%%capture
!pip install git+https://github.com/corentinravoux/flip

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from flip import plot_utils, data_vector, __flip_dir_path__
from flip.covariance import covariance, contraction, fitter

flip_base = Path(__flip_dir_path__)
data_path = flip_base / "data"
plt.style.use(data_path / "style.mplstyle")

In [3]:
grid = pd.read_parquet(data_path / "data_density.parquet")
grid_window = pd.read_parquet(data_path / "data_window_density.parquet")

coordinates_density = np.array([grid["ra"], grid["dec"], grid["rcom_zobs"]])

DataDensity = data_vector.Dens(grid.to_dict(orient='list'))

ktt, ptt = np.loadtxt(data_path / "power_spectrum_tt.txt")
kmt, pmt = np.loadtxt(data_path / "power_spectrum_mt.txt")
kmm, pmm = np.loadtxt(data_path / "power_spectrum_mm.txt")

sigmag_fiducial = 6.0

power_spectrum_dict_bias = {"gg": [[kmm, pmm * np.array(grid_window["window_mm"])**2]]}
power_spectrum_dict = {"gg": [[kmm, pmm * np.array(grid_window["window_mm"])**2],
                              [kmt, pmt * np.array(grid_window["window_mt"])],
                              [ktt, ptt]]}


# Fit fs8 

In [4]:
size_batch=500_000
number_worker=8

covariance_fit = covariance.CovMatrix.init_from_flip(
    "ravouxcarreres",
    "density",
    power_spectrum_dict,
    coordinates_density=coordinates_density,
    size_batch=size_batch,
    number_worker=number_worker,
    additional_parameters_values=(sigmag_fiducial,),
    variant="nobeta",
)


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000030.35 ]: 01-08 09:12  root            INFO     Covariance matrix generated from flip with ravouxcarreres model in 5.06e+00 seconds


In [5]:
likelihood_type="multivariate_gaussian"
likelihood_properties = {"inversion_method": "cholesky_inverse"}

parameter_dict = {"bs8": {"value": 1.0,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "fs8": {"value": 0.4,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                 }




minuit_fitter = fitter.FitMinuit.init_from_covariance(
        covariance_fit,
        DataDensity,
        parameter_dict,
        likelihood_type=likelihood_type,
        likelihood_properties = likelihood_properties)
print('Free parameters are:', minuit_fitter.likelihood.free_par)

minuit_fitter.run()


Free parameters are: ['fs8', 'bs8']


[ 000031.45 ]: 01-08 09:12  root            INFO     ┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 375.7                      │              Nfcn = 48               │
│ EDM = 4.1e-07 (Goal: 0.0001)     │            time = 1.1 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────

{'bs8': 0.8665761269857377, 'fs8': 0.6373948136066049}

# Fit fs8 with an interpolation scheme for sigma_g

In [ ]:
sigmag_list = np.linspace(0.5, 3.0, 10)
covariance_list = []


for sigg in sigmag_list:
    covariance_list.append(
        covariance.CovMatrix.init_from_flip(
            "ravouxcarreres",
            "density",
            power_spectrum_dict,
            coordinates_density=coordinates_density,
            size_batch=size_batch,
            number_worker=number_worker,
            additional_parameters_values=(sigg,),
            variant="nobeta",
        )
    )


[ 000035.68 ]: 01-08 09:12  root            INFO     Covariance matrix generated from flip with ravouxcarreres model in 4.22e+00 seconds
[ 000040.11 ]: 01-08 09:12  root            INFO     Covariance matrix generated from flip with ravouxcarreres model in 4.41e+00 seconds


In [ ]:
parameter_dict = {"bs8": {"value": 1.0,
                          "fixed" : False,
                          "limit_low" : 0.0,
                         },
                  "fs8": {"value": 0.4,
                          "fixed" : False,
                          "limit_low" : 0.0,
                         },
                  "sigg": {"value": 2.0,
                          "limit_low" : 0.5,
                          "limit_up" : 3.0,
                          "fixed" : False,
                         },
                 }    



likelihood_type = "multivariate_gaussian_interp1d"
likelihood_properties = {"inversion_method": "inverse"}



minuit_fitter_interp = fitter.FitMinuit.init_from_covariance(
        covariance_list,
        DataDensity,
        parameter_dict,
        likelihood_type=likelihood_type,
        likelihood_properties = likelihood_properties,
        interpolation_value_name = "sigg",
        interpolation_value_range = sigmag_list,
)

print('Free parameters are:', minuit_fitter_interp.likelihood.free_par)

minuit_fitter_interp.run()


# Some plots

In [ ]:
plot_utils.plot_correlation_from_likelihood(
    minuit_fitter.likelihood,
    parameter_dict, 
    vmin = -0.01, vmax= 0.01
)

In [ ]:
r_perpendicular = np.linspace(-200,200,200)
r_parallel = np.linspace(-200,200,200)

r_reference_parallel = 30 
r_reference_perpendicular = 0 

contraction_plot = contraction.Contraction.init_from_flip(
    "ravouxcarreres",
    "density",
    power_spectrum_dict,
    r_perpendicular,
    r_parallel,
    r_reference_perpendicular,
    r_reference_parallel,
    coordinate_type="rprt",
    additional_parameters_values=(sigmag_fiducial,),
    variant="nobeta"
)

parameter_values_dict = {parameters: parameter_dict[parameters]["value"] for parameters in parameter_dict}


plot_utils.plot_2d_contraction(contraction_plot, 
                    parameter_values_dict, 
                    rs_multiplied = True)
